**DACON 대회 참여 시 참고하기**

1. 데이터 load
2. 데이터 결측값 처리
3. correlation (feature selection)
4. AutoML (model selection) base model + blender = 3+1, 5+1, 7+1, ... (blender = catboost, catboost 2023.04 기준 상위권)
5. 선정된 모델들을 전부 다 optuna로 최적의 hyper를 찾는다.
6. Stacking에 모델을 넣는다. (Ridge Classifier, Logistic Regression, Extra Trees Classifier)

In [ ]:
import pandas as pd

f = pd.read_csv("titanic.csv")
f.isnull().sum()

In [ ]:
f.Age.fillna(f.Age.median(), inplace=True)

# 여자:0 남자:1로 변환
f.Sex = f.Sex.map({"female":0, "male":1})

# Embarked 누락값을 S로
f.Embarked.fillna("S", inplace=True)
embarked_dummies = pd.get_dummies(f.Embarked, prefix="Embarked", drop_first=True)

# 뒤에 붙여준다.
titanic = pd.concat([f, embarked_dummies], axis=1)

In [ ]:
f.corr() # Pclass, Sex, Fare

In [ ]:
!pip install pycaret

In [ ]:
# pycaret classification 모듈
from pycaret.classification import setup

# pycaret classification 모델들을 비교하기 위한 모듈
from pycaret.classification import compare_models

# 컬럼명 String으로 반환
# 데이터셋 설정
grid = setup(data=f, target="Survived",  verbose=True)

# 모델 학습
best = compare_models()

# best 모델 확인
print(best)

In [ ]:
X = f[["Pclass", "Sex", "Fare"]]
y = f.Survived

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2 , random_state=42)

In [ ]:
!pip install optuna

In [ ]:
import optuna
import numpy as np
import sklearn
import xgboost as xgb

def objective(trial):
  params = {
      "max_depth": trial.suggest_int("max_depth", 4, 10),
      "learning_rate": trial.suggest_uniform("learning_rate", 0.0001, 0.99),
      "n_estimators": trial.suggest_int("n_estimators", 1000, 10000, step=100),
  }

  model = xgb.XGBClassifier(**params, random_state = 7)
  bst = model.fit(X_train, y_train, eval_set=[(X_valid,y_valid)], early_stopping_rounds=50, verbose=False)
  preds = bst.predict(X_valid)
  pred_labels = np.rint(preds)
  accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)

  return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)

print("Number of finished trials : ", len(study.trials))
print("Best trial : ")

trial = study.best_trial

In [ ]:
from sklearn.ensemble import StackingClassifier

# base_models 설정
base_models = [
    ('rc', RidgeClassifier()),
    ('lr', LogisticRegression(base_estimator=DecisionTreeClassifier()),
    ('lr', ExtraTreesClassifier(base_estimator=DecisionTreeClassifier())
]

# stacking 설정
clf = StackingClassifier(estimators=base_models, final_estimator=CatboostClassfier())
clf.fit(X_train,y_train)

In [ ]:
# 대회에 제출할 csv 파일을 생성